# i.MX RT118x External Memory Configuration with SPSDK


**Procedure of initialization and using external memory connected MCU over blhost interface.**
1. Create the flashloader application with mBoot(blhost application) 
2. Load the flashloader application to the target device
3. Generate a blhost script for memory configuration
4. Execute the blhost script to configure the external memory
   - Example of creation a non-default memory configuration
5. Build a bootable image and load it into a memory configured though the blhost-script


## 1. Prerequisites

 SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)

 ### 1.1 Connect your board to you PC.
Use both the Debug USB Port and USB OTG1 port and set the board to SDP mode.
Configure Boot Mode Switch to SDP: 1-OFF, 2-OFF, 3-OFF, 4-ON\
Connect your board to your PC via a micro USB using the USB OTG1 port - J33\
Connect your board to your PC via a micro USB using the Debug USB port - J53

**The following picture describes connector placement of RT1180 EVK:**

<img src="../../_data/img/boards/rt1189-evk.png" alt="rt1189-evk" height="400">
    

Run the code below to initialize parameters for the script:

In [1]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
# Execute this cell to enable execution of the ! line magic
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [2]:
import os

WORKSPACE = "workspace/"  # change this to path to your workspace
INPUTS = "inputs/"
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

FLASHLOADER_FILE = (
    "../../ahab/rt118x_signed_flashloader/workspace/flashloader.bin"  # Path to Flashloader
)
CFG_MEM_FILE = WORKSPACE + "config_mem.bls"  # Configure memory template
CUSTOM_CFG_MEM_FILE = WORKSPACE + "custom_config_mem.bls"  # Configure memory template
FCB_FILE = WORKSPACE + "fcb.bin"  # Firmware Configuration block file
CUSTOM_FCB_FILE = WORKSPACE + "custom_fcb.bin"  # Firmware Configuration block file

## 2. Create the flashloader application with mBoot(blhost application) 
We will need program our boards with a flashloader binary to be able to send blhost commands to it. To get such a binary, we can use the [signed_flashloader notebook](../../ahab/rt118x_signed_flashloader/rt118x_signed_flashloader.ipynb). The details are out of scope for this session, so just open the [signed_flashloader notebook](../../ahab/rt118x_signed_flashloader/rt118x_signed_flashloader.ipynb), use the "Run All Cells" option and close it again.

![Run All Cells](./img/run_all_cells.png "run all cells")

Scan for available devices with `nxpdevscan`, connect to the i.MX RT1180 and load a flashloader into it using `blhost`

Call `nxpdevscan --help` to get a list of options supported by `nxpdevscan`:

In [3]:
%! nxpdevscan

nxpdevscan 
-------- Connected NXP USB Devices --------

OC Blank RT Family - NXP SEMICONDUCTORS
Vendor ID: 0x1fc9
Product ID: 0x014c
Path: HID\VID_1FC9&PID_014C\8&2CB1D4EA&0&0000
Path Hash: cd2d117a
Name: mimxrt1181 | mimxrt1189 | mimxrt1182 | mimxrt1187
Serial number: MCU HID GENERIC DEVICE

-------- Connected NXP UART Devices --------

-------- Connected NXP SIO Devices --------

-------- Connected NXP UUU Devices --------



**The Vendor and Product ID values of the `OO Blank RT Family - NXP SEMICONDUCTORS` are the default values for the boot ROM.**

**Store them in a variable for later use:**

In [4]:
COMPAR = "-u"  # USB
BLHOST_CONNECT_ROM = "0x1fc9,0x014c"  # PID/VID of ROM


**Reset your board by pressing the `SW3` button. This is necessary because calling `nxpdevscan` causes the boot ROM to stop being visible. You can test this by executing the `nxpdevscan` cell one more time.**

Ping the boot ROM via `blhost`:


In [5]:
%! blhost $COMPAR $BLHOST_CONNECT_ROM get-property 1

blhost -u 0x1fc9,0x014c get-property 1 
Response status = 0 (0x0) Success.
Response word 1 = 1258487809 (0x4b030001)
Current Version = K3.0.1


**Load a flashloader image into the device:**

In [6]:
from time import sleep

assert os.path.exists(FLASHLOADER_FILE)

%! blhost $COMPAR $BLHOST_CONNECT_ROM load-image $FLASHLOADER_FILE
sleep(3)  # wait for device

blhost -u 0x1fc9,0x014c load-image ../../ahab/rt118x_signed_flashloader/workspace/flashloader.bin 
Loading image
Response status = 0 (0x0) Success.


Call `nxpdevscan` again to get the Value and Product ID of the flashloader:

In [7]:
%! nxpdevscan

nxpdevscan 
-------- Connected NXP USB Devices --------

USB COMPOSITE DEVICE - FREESCALE SEMICONDUCTOR INC.
Vendor ID: 0x15a2
Product ID: 0x0073
Path: HID\VID_15A2&PID_0073\9&1977C631&0&0000
Path Hash: 127da8e0
Name: mcxc242 | mcxc144 | mimxrt1165 | mimxrt1015 | mwct2d17s | mcxc143 | mimxrt1166 | mimxrt1172 | mcxc244 | mimxrt1010 | mcxc444 | mimxrt1176 | mimxrt1173 | mimxrt1181 | mcxe247 | mimxrt1046 | mimxrt1175 | mimxrt1050 | mimxrt1040 | mimxrt1064 | mimxrt1182 | mimxrt1060 | mcxc141 | mimxrt1020 | mwct2015s | mwct2d16s | mcxc142 | mwct2014s | mimxrt1043 | mcxc443 | mwct2016s | mimxrt1189 | mimxrt1187 | mimxrt1024 | mimxrt1171 | mcxc243
Serial number: 

-------- Connected NXP UART Devices --------

-------- Connected NXP SIO Devices --------

-------- Connected NXP UUU Devices --------



**Store the values in a variable again:**

In [8]:
BLHOST_CONNECT_FLDR = "0x15A2,0x0073"  # PID/VID of NXP Flashloader


Reset your board by pressing the `SW3` button. This is necessary because calling `nxpdevscan` causes the flashloader to stop being visible as well. You can test this by executing the `nxpdevscan` cell one more time.

**Reload the flashloader to be able to continue:**

In [21]:
%! blhost $COMPAR $BLHOST_CONNECT_ROM load-image $FLASHLOADER_FILE
sleep(3)  # wait for device

blhost -u 0x1fc9,0x014c load-image ../../ahab/rt118x_signed_flashloader/workspace/flashloader.bin 
Loading image
Response status = 0 (0x0) Success.


**Ping the flashloader to double check everything worked correctly:**

In [10]:
%! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR get-property 1

blhost -v -u 0x15A2,0x0073 get-property 1 
INFO:spsdk.mboot.mcuboot:Connect: identifier='usb', device=USB COMPOSITE DEVICE (0x15A2, 0x0073)path=b'\\\\?\\hid#vid_15a2&pid_0073#9&1977c631&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}' sn=''
INFO:libusbsio.hidapi.dev:Opening HID device at path: 'b'\\\\?\\hid#vid_15a2&pid_0073#9&1977c631&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}''
INFO:libusbsio.hidapi.dev:HID device 3176007822144 is now open
INFO:spsdk.mboot.mcuboot:CMD: GetProperty(CurrentVersion, index=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response word 1 = 1258424320 (0x4b020800)
Current Version = K2.8.0
INFO:spsdk.mboot.mcuboot:Closing: identifier='usb', device=USB COMPOSITE DEVICE (0x15A2, 0x0073)path=b'\\\\?\\hid#vid_15a2&pid_0073#9&1977c631&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}' sn=''
INFO:libusbsio.hidapi.dev:HID device 3176007822144 closed


**Check the enabled flashloader commands:**

In [11]:
%! blhost $COMPAR $BLHOST_CONNECT_FLDR get-property 7

blhost -u 0x15A2,0x0073 get-property 7 
Response status = 0 (0x0) Success.
Response word 1 = 25530335 (0x1858fdf)
Available Commands = ['FlashEraseAll', 'FlashEraseRegion', 'ReadMemory', 'WriteMemory', 'FillMemory', 'GetProperty', 'ReceiveSBFile', 'Execute', 'Call', 'Reset', 'SetProperty', 'FlashReadResource', 'ConfigureMemory', 'GenerateKeyBlob', 'UpdateLifeCycle', 'EleMessage']



## 3 Generate a blhost script for memory configuration using **nxpmemcfg**

**Call `nxpmemcfg` or `nxpmemcfg --help` to get a list of options and sub-commands supported by `nxpmemcfg`**

In [12]:
%! nxpmemcfg

nxpmemcfg 
Usage: nxpmemcfg [OPTIONS] COMMAND [ARGS]...

  Collection of utilities for memory configuration operations.

Options:
  -v, --verbose  Print more detailed information
  -vv, --debug   Display more debugging information.
  --version      Show the version and exit.
  --help         Show this message and exit.

Commands:
  nxpmemcfg          Collection of utilities for memory configuration operations.
  ├── blhost-script  Export the configuration option words to blhost script.
  ├── export         Export the configuration option words from configuration.
  ├── family-info    List known memory configurations for the family.
  ├── get-families   Shows the full families information for commands in this group.
  ├── get-templates  Create template of Memory option words in YAML format.
  └── parse          Parse the existing memory configuration option words.


**Call `nxpmemcfg blhost-script` or `nxpmemcfg blhost-script --help` to get a list of options supported by `nxpmemcfg blhost-script`:**

In [13]:
%! nxpmemcfg blhost-script --help

nxpmemcfg blhost-script --help 

Help for nested command: 'blhost-script'
Command Hierarchy: nxpmemcfg ▶ blhost-script

────────────────────── Root command (nxpmemcfg) help ──────────────────────

Options:
  -v, --verbose  Print more detailed information
  -vv, --debug   Display more debugging information.
  --version      Show the version and exit.
  --help         Show this message and exit.

─────────────────────── Command (blhost-script) help ───────────────────────
Export the configuration option words to blhost script.
Usage: nxpmemcfg blhost-script [OPTIONS]

Options:
  Known chip select. Alternative to known chip is YAML configuration '-c': [all_or_none]
    -f, --family [mimxrt1060|mimxrt1166|mimxrt735s|mcxn547..., and more. Use 'get-families' command to show all.]
                                  Select the chip family.
    -p, --peripheral [flexspi_nor|xspi_nor|flexspi_nand|semc_nor|semc_nand|spi_nor|mmc|sd]
                                  Choose the peripheral of the inp

**Call `nxpmemcfg family-info` to check the known memories and their default configurations:**

In [14]:
%! nxpmemcfg family-info -f mimxrt1189

nxpmemcfg family-info -f mimxrt1189 
List of all supported peripherals and its instances:
╔═══╦══════════════════════════════╦═════════════╦══════════╦══════════════╦══════════╦═══════════╦══════════════╦════════╦════════╗
║ # ║            Family            ║ flexspi_nor ║ xspi_nor ║ flexspi_nand ║ semc_nor ║ semc_nand ║   spi_nor    ║  mmc   ║   sd   ║
╠═══╬══════════════════════════════╬═════════════╬══════════╬══════════════╬══════════╬═══════════╬══════════════╬════════╬════════╣
║ 0 ║ mimxrt1189, Revision: latest ║    [1, 2]   ║   N/A    ║    [1, 2]    ║   Yes    ║    Yes    ║ [1, 2, 4, 5] ║ [1, 2] ║ [1, 2] ║
╚═══╩══════════════════════════════╩═════════════╩══════════╩══════════════╩══════════╩═══════════╩══════════════╩════════╩════════╝
List of all known memory configuration option words:
╔════╦══════╦════════════════╦══════════════╦═════════════╦════════════════════════════════════╦════════╗
║ #  ║ Type ║      Name      ║ Manufacturer ║  Interface  ║ Option words              

**Since we're using the EVK, we are interested in the default FlexSPI NOR from Winbond on interface index 1:**

```
╔════╦══════════╦═════════════╦══════════════╦══════════╦═══════════╦════════════════════╦════════╦════════╗
║ #  ║  Family  ║ flexspi_nor ║ flexspi_nand ║ semc_nor ║ semc_nand ║      spi_nor       ║  mmc   ║   sd   ║
╠════╬══════════╬═════════════╬══════════════╬══════════╬═══════════╬════════════════════╬════════╬════════║
║ 15 ║  rt118x  ║    [1, 2]   ║    [1, 2]    ║   Yes    ║    Yes    ║ [0, 1, 2, 3, 4, 5] ║ [0, 1] ║ [0, 1] ║
╚════╩══════════╩═════════════╩══════════════╩══════════╩═══════════╩════════════════════╩════════╩════════╝

╔════╦══════════════╦══════════════╦══════════════════════╦═════════════╦════════════════════════════════════╗
║ #  ║  Peripheral  ║ Manufacturer ║         Name         ║  Interface  ║ Option words                       ║
╠════╬══════════════╬══════════════╬══════════════════════╬═════════════╬════════════════════════════════════╣
║ 0  ║ flexspi_nor  ║   Winbond    ║      W25QxxxJV       ║   quad_spi  ║ Opt0: 0xC0000207                   ║
╚════╩══════════════╩══════════════╩══════════════════════╩═════════════╩════════════════════════════════════╝
```

**Note that if you use this approach, you will configure your memory with the default option words shown in the table.**

Now we can set the options according to the table:
- f: device family => **mimxrt1189**
- m: memory chip => **W25QxxxJV**
- i: memory interface => **quad_spi**
- ix: peripheral instance index => **1**

In [15]:
%! nxpmemcfg blhost-script -f mimxrt1189 -m W25QxxxJV -p flexspi_nor -i quad_spi -ix 1 -o $CFG_MEM_FILE --force --fcb $FCB_FILE

assert os.path.exists(CFG_MEM_FILE)

nxpmemcfg blhost-script -f mimxrt1189 -m W25QxxxJV -p flexspi_nor -i quad_spi -ix 1 -o workspace/config_mem.bls --force --fcb workspace/fcb.bin 
Loaded option words: Opt0: 0xC0000007
Exported blhost script.


#### Let's look inside the blhost-script:
```
# BLHOST configure memory programming script
# Generated by SPSDK NXPMEMCFG tool
# Chip: mimxrt1189
# Peripheral: flexspi_nor
# Instance: 1

# Switch the instance of the peripheral to 1:
fill-memory 0x1FFE0000 4 0xCF900001
configure-memory 9 0x1FFE0000

# Configure memory:
# Option word 0: 0xC0000207
fill-memory 0x1FFE0000 4 0xC0000207
configure-memory 9 0x1FFE0000

# Script to erase FCB location, create FCB and read back a FCB block:
flash-erase-region 0x28000000 0x1000
fill-memory 0x1FFE0000 4 0xF000000F
configure-memory 9 0x1FFE0000
read-memory 0x28000400 0x200 workspace/fcb.bin

```

---
## 4. Execute the blhost script

The blhost script is used to configure the memory controller and initialize the external memory on the i.MX RT1180 EVK board. The script utilizes the NXP MCU Bootloader Host (blhost) tool to communicate with the target device over a serial or USB connection. Due to request to get FCB block, the script will erase the first memory block, generate a new FCB by BLHOST command and read back the FCB block to file.

In [16]:
# The board must be in SDP mode
# Configure Boot Mode Switch to : 1/2/3-OFF, 4-ON
# Connect micro USB cable into USB OTG1         - J33
# Connect micro USB cable into Debug USB Port   - J53

# FLASHLOADER_FILE = "../flashloader/ahab/workspace/flashloader.bin"  # Path to Flashloader
# CFG_MEM_FILE = WORKSPACE + "config_mem.bls"                         # Configure memory template
# FCB_FILE = WORKSPACE + "fcb.bin"                                    # Firmware Configuration block file

# COMPAR = "-u"                          # USB
# BLHOST_CONNECT_ROM = "0x1fc9,0x014c"   # PID/VID of ROM
# BLHOST_CONNECT_FLDR = "0x15A2,0x0073"  # PID/VID of NXP Flashloader

# Execute the commands generated by "blhost-script". This script will configure flash,
# program FCB into the external memory and reads back the binary data. Be aware tha erase of the first sector must be done.
# The script can be executed only once to read FCB.
%! blhost $COMPAR $BLHOST_CONNECT_FLDR batch $CFG_MEM_FILE

blhost -u 0x15A2,0x0073 batch workspace/config_mem.bls 
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Reading memory
Response status = 0 (0x0) Success.
Response word 1 = 512 (0x200)
Read 512 of 512 bytes.


---
## 5. Create a non-default memory configuration

The `nxpmemcfg get-templates` sub-command generates YAML configuration templates that serve as a starting point for customization. These templates can be easily modified to meet specific memory configuration requirements for the MIMXRT1189 microcontroller.

In [17]:
%! nxpmemcfg get-templates

nxpmemcfg get-templates 

Help for nested command: 'get-templates'
Command Hierarchy: nxpmemcfg ▶ get-templates

────────────────────── Root command (nxpmemcfg) help ──────────────────────

Options:
  -v, --verbose  Print more detailed information
  -vv, --debug   Display more debugging information.
  --version      Show the version and exit.
  --help         Show this message and exit.

─────────────────────── Command (get-templates) help ───────────────────────
Create template of Memory option words in YAML format.
Usage: nxpmemcfg get-templates [OPTIONS]

Options:
  -f, --family [mimxrt1171|mimxrt1015|lpc5528|lpc5504..., and more. Use 'get-families' command to show all.]
                                  [required] Select the chip family.
  -r, --revision TEXT             Chip revision; if not specified, most recent
                                  one will be used
  -o, --output DIRECTORY          Path to a directory, where to store
                                  generated/pars

### 5.1 Let's inspect the **flexspi_nor** template

Let's examine the **flexspi_nor** template. When inspecting this template, it's important to note that the 'each' value can be modified using the options provided in the comments. Additionally, if a configuration line is removed, the default value will be automatically applied. These features provide flexibility in customization while ensuring that the configuration remains functional by falling back to default values when necessary.

In [18]:
# Show the typical external memory configuration for FlexSPI and NOR
YamlDiffWidget("inputs/rt118x_external_memory_config.diffc").html

nxpmemcfg get-templates -f mimxrt1189 -o workspace/cfg_opt_templates/ --force 
The Memory Configuration template has been saved into 'workspace/cfg_opt_templates/ow_flexspi_nor.yaml' YAML file
The Memory Configuration template has been saved into 'workspace/cfg_opt_templates/ow_flexspi_nand.yaml' YAML file
The Memory Configuration template has been saved into 'workspace/cfg_opt_templates/ow_semc_nor.yaml' YAML file
The Memory Configuration template has been saved into 'workspace/cfg_opt_templates/ow_semc_nand.yaml' YAML file
The Memory Configuration template has been saved into 'workspace/cfg_opt_templates/ow_spi_nor.yaml' YAML file
The Memory Configuration template has been saved into 'workspace/cfg_opt_templates/ow_mmc.yaml' YAML file
The Memory Configuration template has been saved into 'workspace/cfg_opt_templates/ow_sd.yaml' YAML file


**Once the template is configured for the new memory, it can be used to export a blhost script with `nxpmemcfg blhost-script` like this:**

In [19]:
%! nxpmemcfg blhost-script -c "inputs/ow_flexspi_nor.yaml" -ix 1 -o $CUSTOM_CFG_MEM_FILE --force

nxpmemcfg blhost-script -c "inputs/ow_flexspi_nor.yaml" -ix 1 -o workspace/custom_config_mem.bls --force 
Exported blhost script.


**Let's inspect the blhost-script generated from the `flexspi_nor` template (note the difference in the option word):**

```
# BLHOST configure memory programming script
# Generated by SPSDK NXPMEMCFG tool
# Chip: mimxrt1189
# Peripheral: flexspi_nor
# Instance: 1

# Switch the instance of the peripheral to 1:
fill-memory 0x1FFE0000 4 0xCF900001
configure-memory 9 0x1FFE0000

# Configure memory:
# Option word 0: 0xC0000000
fill-memory 0x1FFE0000 4 0xC0000000
configure-memory 9 0x1FFE0000
```

## 6. Build a bootable image

Load it into a memory configured though the blhost-script.

This process is explained in the [Secure Boot notebook](../../ahab/rt118x_secure_boot/rt118x_secure_boot.ipynb)